In [1]:
import pandas as pd
import os
import torch
from model.database_util import *

In [2]:
import torch
from model.util import Normalizer

# cost_norm = Normalizer(1, 100)
# cost_norm = Normalizer(-3.61192, 12.290855)
#cost_norm = Normalizer(5, 2611)
cost_norm = Normalizer(8.26, 11.12)

## Define Model

In [3]:
class Args:
    # bs = 1024
    # SQ: smaller batch size
    bs = 2
    #lr = 0.001
    lr = 0.001
    # epochs = 200
    epochs = 50
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    # device = 'cuda:0'
    device = 'cpu'
    newpath = 'job_queries_training'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [4]:
from model.model import QueryFormer

model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = False, use_hist = False, \
                 pred_hid = args.pred_hid
                )

In [5]:
from model.dataset import PlanTreeDataset

# Loading tensors

In [6]:
# Load the saved tensor collection
loaded_tensors = torch.load("./job_queries/tensors/query_1_2024-12-03-20.43.44.200877.pt")

# Access each tensor
x_loaded = loaded_tensors["x"]
rel_pos_loaded = loaded_tensors["rel_pos"]
attn_bias_loaded = loaded_tensors["attn_bias"]
heights_loaded = loaded_tensors["heights"]
cost_labels_loaded = loaded_tensors["cost_labels"]
raw_costs_loaded = loaded_tensors["raw_costs"]

# Print the shapes to verify
print(f"x shape: {x_loaded.shape}")
print(f"rel_pos shape: {rel_pos_loaded.shape}")
print(f"attn_bias shape: {attn_bias_loaded.shape}")
print(f"heights shape: {heights_loaded.shape}")
print(f"cost_labels shape: {cost_labels_loaded.shape}")
print(f"raw_costs shape: {raw_costs_loaded.shape}")

x shape: torch.Size([1, 30, 1161])
rel_pos shape: torch.Size([1, 30, 30])
attn_bias shape: torch.Size([1, 31, 31])
heights shape: torch.Size([1, 30])
cost_labels shape: torch.Size([1])
raw_costs shape: torch.Size([1])


In [7]:
x_loaded

tensor([[[1., 0., 0.,  ..., 1., 1., 1.],
         [2., 0., 0.,  ..., 1., 1., 1.],
         [3., 1., 0.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])

In [8]:
rel_pos_loaded

tensor([[[ 1,  2,  3,  4,  4,  5,  5,  6,  6,  7,  8,  8,  9, 10, 10, 11, 11,
           9,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61,  1,  2,  3,  3,  4,  4,  5,  5,  6,  7,  7,  8,  9,  9, 10, 10,
           8,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61,  1,  2,  2,  3,  3,  4,  4,  5,  6,  6,  7,  8,  8,  9,  9,
           7,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61,  1, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
          61, 61,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61, 61,  1,  2,  2,  3,  3,  4,  5,  5,  6,  7,  7,  8,  8,
           6,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61, 61, 61,  1, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
          61, 61,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61, 61, 61, 61,  1,  2,  2,  3,  4,  4,  5,  6,  6,  7,  7,
           5,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61,

In [9]:
attn_bias_loaded

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, -inf, 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
       

In [10]:
heights_loaded

tensor([[12, 11, 10,  2,  9,  2,  8,  2,  7,  6,  2,  2,  5,  4,  2,  2,  3,  2,
          2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [11]:
raw_costs_loaded

tensor([66844.], dtype=torch.float64)

In [12]:
cost_labels_loaded

tensor([0.9965], dtype=torch.float64)

In [13]:
# Print the shapes to verify
print(f"x shape: {x_loaded.shape}")
print(f"rel_pos shape: {rel_pos_loaded.shape}")
print(f"attn_bias shape: {attn_bias_loaded.shape}")
print(f"heights shape: {heights_loaded.shape}")
print(f"cost_labels shape: {cost_labels_loaded.shape}")
print(f"raw_costs shape: {raw_costs_loaded.shape}")

x shape: torch.Size([1, 30, 1161])
rel_pos shape: torch.Size([1, 30, 30])
attn_bias shape: torch.Size([1, 31, 31])
heights shape: torch.Size([1, 30])
cost_labels shape: torch.Size([1])
raw_costs shape: torch.Size([1])


## TEST - Loading 2 tensors

In [14]:
# Load the saved tensor collection
loaded_tensors1 = torch.load("./job_queries/tensors/query_1_2024-12-03-20.43.44.200877.pt")

# Access each tensor
x_loaded1 = loaded_tensors1["x"]
rel_pos_loaded1 = loaded_tensors1["rel_pos"]
attn_bias_loaded1 = loaded_tensors1["attn_bias"]
heights_loaded1 = loaded_tensors1["heights"]
cost_labels_loaded1 = loaded_tensors1["cost_labels"]
raw_costs_loaded1 = loaded_tensors1["raw_costs"]

# Load the saved tensor collection
loaded_tensors2 = torch.load("./job_queries/tensors/query_2_2024-12-03-20.43.45.954590.pt")

# Access each tensor
x_loaded2 = loaded_tensors2["x"]
rel_pos_loaded2 = loaded_tensors2["rel_pos"]
attn_bias_loaded2 = loaded_tensors2["attn_bias"]
heights_loaded2 = loaded_tensors2["heights"]
cost_labels_loaded2 = loaded_tensors2["cost_labels"]
raw_costs_loaded2 = loaded_tensors2["raw_costs"]

x_list = [x_loaded1, x_loaded2]
rel_pos_list = [rel_pos_loaded1, rel_pos_loaded2]
attn_bias_list = [attn_bias_loaded1, attn_bias_loaded2]
heights_list = [heights_loaded1, heights_loaded2]
cost_labels_list = [cost_labels_loaded1, cost_labels_loaded2]
raw_costs_list = [raw_costs_loaded1, raw_costs_loaded2]

dataset = PlanTreeDataset(
    2, # number of training examples 
    x_list,
    attn_bias_list,
    rel_pos_list,
    heights_list,
    cost_labels_list,
    raw_costs_list
)

print("Dataset length:", len(dataset))
print("First sample:", dataset[0])

Dataset length: 2
First sample: ({'x': tensor([[[1., 0., 0.,  ..., 1., 1., 1.],
         [2., 0., 0.,  ..., 1., 1., 1.],
         [3., 1., 0.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]]), 'attn_bias': tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0

In [15]:
# Get the length of the dataset
print(f"Dataset size: {len(dataset)}")

# Access a single sample
sample, label = dataset[0]

# Print the sample contents
print("Sample contents:")
print("Feature Matrix (x):", sample['x'].shape)
print("Attention Bias (attn_bias):", sample['attn_bias'].shape)
print("Relative Positions (rel_pos):", sample['rel_pos'].shape)
print("Heights (heights):", sample['heights'].shape)
print("Label:", label)

Dataset size: 2
Sample contents:
Feature Matrix (x): torch.Size([1, 30, 1161])
Attention Bias (attn_bias): torch.Size([1, 31, 31])
Relative Positions (rel_pos): torch.Size([1, 30, 30])
Heights (heights): torch.Size([1, 30])
Label: (tensor([0.9965], dtype=torch.float64), tensor([66844.], dtype=torch.float64))


In [16]:
# Example numpy label
import numpy as np
import torch.nn as nn
import importlib

from model import trainer
importlib.reload(trainer)
from  model.trainer import train_single, train


crit = nn.MSELoss()

# Train the model with the numpy label
# trained_model = train_single(model, dataset, dataset, crit, cost_norm, args)
model, best_model_path, train_embeddings, val_embeddings = train(model, dataset, dataset, crit, cost_norm, args)


pred_output shape: torch.Size([2, 1])
Cost Predictions Shape: torch.Size([2])
Batch Cost Label Shape: torch.Size([2])
cost_preds: tensor([0.5416, 0.5031], grad_fn=<SqueezeBackward1>)
batch_cost_label: tensor([0.9966, 0.9965])
Epoch: 0  Avg Loss: 0.11264064908027649, Time: 1.23982834815979
Median: 3.8878654855117554
Mean: 3.8878654855117554
pred_output shape: torch.Size([2, 1])
Cost Predictions Shape: torch.Size([2])
Batch Cost Label Shape: torch.Size([2])
cost_preds: tensor([0.9912, 0.9917], grad_fn=<SqueezeBackward1>)
batch_cost_label: tensor([0.9965, 0.9966])
pred_output shape: torch.Size([2, 1])
Cost Predictions Shape: torch.Size([2])
Batch Cost Label Shape: torch.Size([2])
cost_preds: tensor([0.9968, 0.9966], grad_fn=<SqueezeBackward1>)
batch_cost_label: tensor([0.9965, 0.9966])
pred_output shape: torch.Size([2, 1])
Cost Predictions Shape: torch.Size([2])
Batch Cost Label Shape: torch.Size([2])
cost_preds: tensor([0.9983, 0.9984], grad_fn=<SqueezeBackward1>)
batch_cost_label: tenso

/home/db2inst1/learnedwmp/templates/job-templatizing-queryformer-modified/model/trainer.py:46: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(np.log(ps), np.log(ls))


pred_output shape: torch.Size([2, 1])
ps: [66869.125 66869.125]
ls: [tensor([66844.], dtype=torch.float64), tensor([66860.], dtype=torch.float64)]
len(ls): 2
pred_output shape: torch.Size([2, 1])
Cost Predictions Shape: torch.Size([2])
Batch Cost Label Shape: torch.Size([2])
cost_preds: tensor([0.9965, 0.9966], grad_fn=<SqueezeBackward1>)
batch_cost_label: tensor([0.9966, 0.9965])
pred_output shape: torch.Size([2, 1])
ps: [66884.76 66884.76]
ls: [tensor([66844.], dtype=torch.float64), tensor([66860.], dtype=torch.float64)]
len(ls): 2
pred_output shape: torch.Size([2, 1])
Cost Predictions Shape: torch.Size([2])
Batch Cost Label Shape: torch.Size([2])
cost_preds: tensor([0.9972, 0.9968], grad_fn=<SqueezeBackward1>)
batch_cost_label: tensor([0.9966, 0.9965])
pred_output shape: torch.Size([2, 1])
ps: [66887.945 66888.01 ]
ls: [tensor([66844.], dtype=torch.float64), tensor([66860.], dtype=torch.float64)]
len(ls): 2
pred_output shape: torch.Size([2, 1])
Cost Predictions Shape: torch.Size([2]

In [17]:
print(f"Number of training embeddings (best epoch): {len(train_embeddings)}")
print(f"First training embedding shape: {train_embeddings[0].shape}")
print(f"Number of validation embeddings (best epoch): {len(val_embeddings)}")
print(f"First validation embedding shape: {val_embeddings[0].shape}")


Number of training embeddings (best epoch): 2
First training embedding shape: (1417,)
Number of validation embeddings (best epoch): 2
First validation embedding shape: (1417,)


In [18]:
print(f"First validation embedding shape: {val_embeddings[0]}")

First validation embedding shape: [-1.031179   6.3488293  9.419984  ... 14.152625  -3.491641   3.5418007]


In [19]:
type(val_embeddings[0])

numpy.ndarray

In [20]:
print(f"First validation embedding shape: {val_embeddings[1]}")

First validation embedding shape: [-1.0309108  6.349203   9.420137  ... 14.152448  -3.492036   3.541753 ]


In [21]:
len(val_embeddings[0])

1417

In [22]:
import numpy as np

# Disable truncation
np.set_printoptions(threshold=np.inf, linewidth=1000)

for i, emb in enumerate(val_embeddings):
    print(f"Full Embedding {i}:\n{emb}")


Full Embedding 0:
[-1.03117895e+00  6.34882927e+00  9.41998386e+00 -3.90928221e+00  1.19157000e+01 -4.41831064e+00 -4.93687630e+00  7.61633444e+00  5.80436993e+00  8.62499714e+00 -6.53570175e+00 -1.01669443e+00 -7.70654678e+00  1.25583693e-01 -1.79030740e+00 -3.69698000e+00  3.85393143e+00 -3.38230073e-01 -7.21047544e+00 -3.81462550e+00  7.42372894e+00  1.04346838e+01 -6.55981588e+00 -7.76527071e+00  7.35305548e+00 -9.54584503e+00 -3.08760428e+00  2.51115704e+00  5.99572754e+00 -5.49904776e+00  2.72151899e+00  5.53854084e+00 -4.44113255e+00  8.77591419e+00  5.81835938e+00  5.76462507e+00  6.33167505e+00  9.28437948e-01  1.02848399e+00  2.37525880e-01 -4.31308460e+00 -1.18324304e+00  9.76466560e+00  4.99428988e+00 -6.50026751e+00 -1.14985061e+00  4.78108168e+00  8.52582741e+00 -2.12309241e+00 -2.19312608e-01  3.42280817e+00 -1.48343205e+00  2.83918977e+00 -4.97438335e+00 -5.27138281e+00 -6.36840343e+00  2.01121664e+00 -4.98556566e+00  7.60432899e-01 -6.79205561e+00  1.36579502e+00  1.92